In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
filtrar_portfolio.py

Script 1 para:
  1) Ler CSV "Portfolio_metadados.csv";
  2) Remover duplicados (baseado no título normalizado);
  3) Filtrar registros sem Abstract, Affiliations ou Author Keywords;
  4) Reordenar colunas e salvar em Excel "Portfolio_analitico.xlsx";
  5) Exibir métricas de limpeza, data/hora e tempo de execução.

USO NO CLUSTER UFSC (JupyterLab ou SLURM):
  • Coloque este script e o CSV na mesma pasta;
  • No terminal do vLab:
      module load python/3.10 pandas openpyxl
      python filtrar_portfolio.py

IMPORTANTE (Ciência Aberta):
  • Entrada: "Portfolio_metadados.csv" no mesmo diretório.
  • Saída : "Portfolio_analitico.xlsx" no mesmo diretório.

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado  
Data: 01-mai-2025
"""

import sys
import subprocess
import re
import unicodedata
from pathlib import Path
from collections import Counter
from datetime import datetime
from zoneinfo import ZoneInfo

# ─── 0) marca data/hora de início ──────────────────────────
TZ = ZoneInfo("America/Sao_Paulo")
inicio = datetime.now(TZ)
print(f"▶️ Início em      : {inicio:%d/%m/%Y %H:%M:%S} (Flonianópolis | SC)")

# 1) Instalar dependências se faltarem
def instalar(pkg):
    try:
        __import__(pkg)
    except ImportError:
        print(f"⚙️ Instalando {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        import site; site.addsitedir(site.USER_SITE)

for pkg in ("pandas","openpyxl"):
    instalar(pkg)

import pandas as pd

# ——————— FUNÇÕES AUXILIARES ———————
def normalizar(texto: str) -> str:
    if not isinstance(texto, str):
        return ""
    nfkd = unicodedata.normalize("NFKD", texto)
    s = "".join(c for c in nfkd if not unicodedata.combining(c)).lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def formata_numero(n, decimals=0) -> str:
    s = f"{n:,.{decimals}f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

# ——————— CONFIGURAÇÃO DE CAMINHOS ———————
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path.cwd()

csv_path    = base_dir / "Portfolio_metadados.csv"
output_path = base_dir / "Portfolio_analitico.xlsx"

# ——————— VERIFICAÇÃO DE EXISTÊNCIA ———————
if not csv_path.exists():
    sys.exit(f"⛔ Arquivo não encontrado: {csv_path}")

# ——————— LEITURA E FILTRAGEM ———————
df = pd.read_csv(csv_path)
total_original = len(df)

# 1) Duplicados (título normalizado)
df["Title_norm"] = df["Title"].apply(normalizar)
mask_dup = df.duplicated(subset="Title_norm", keep="first")
n_duplicados = int(mask_dup.sum())
df = df[~mask_dup].reset_index(drop=True)

# 2) Sem Abstract
n_sem_abstract = int(df["Abstract"].isna().sum())
df = df.dropna(subset=["Abstract"]).reset_index(drop=True)

# 3) Sem Affiliations
n_sem_affiliacao = int(df["Affiliations"].isna().sum())
df = df.dropna(subset=["Affiliations"]).reset_index(drop=True)

# 4) Sem Author Keywords
n_sem_keywords = int(df["Author Keywords"].isna().sum())
df = df.dropna(subset=["Author Keywords"]).reset_index(drop=True)

total_filtrado = len(df)

# ——————— REORDENAÇÃO DAS COLUNAS ———————
ordem = [
    "Year","Cited by","Title","Abstract",
    "Affiliations","Source title","Author Keywords"
]
df = df[ordem]

# ——————— EXPORTAÇÃO PARA EXCEL ———————
df.to_excel(output_path, index=False)

# ─── FIM DA CONTAGEM DO TEMPO ───────────────────────────────
fim = datetime.now(TZ)
dur = fim - inicio
dur_str = str(dur).split(".")[0]

# ─── SAÍDA DAS MÉTRICAS ────────────────────────────────────
print("\n📋 Métricas Gerais:")
print(f"   Total originais           : {formata_numero(total_original)}")
print(f"   Duplicados removidos      : {formata_numero(n_duplicados)}")
print(f"   Sem Abstract              : {formata_numero(n_sem_abstract)}")
print(f"   Sem Affiliations          : {formata_numero(n_sem_affiliacao)}")
print(f"   Sem Keywords              : {formata_numero(n_sem_keywords)}")
print(f"   Total após filtragem      : {formata_numero(total_filtrado)}")

print("\n" + "="*55)
print(f"⏱️ Tempo de execução: {dur_str}")
print(f"📅 Finalizado em: {fim:%d/%m/%Y %H:%M:%S} (Florianópolis | SC)")
print("="*55)
print(f"→ Arquivo salvo em: {output_path}\n")

▶️ Início em      : 03/05/2025 09:06:34 (Flonianópolis | SC)

📋 Métricas Gerais:
   Total originais           : 15.922
   Duplicados removidos      : 75
   Sem Abstract              : 0
   Sem Affiliations          : 412
   Sem Keywords              : 1.821
   Total após filtragem      : 13.614

⏱️ Tempo de execução: 0:00:04
📅 Finalizado em: 03/05/2025 09:06:39 (Florianópolis | SC)
→ Arquivo salvo em: /home/jovyan/Congresso UFSC2025/Portfolio_analitico.xlsx



In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
keyword_analitico.py

Script 2 para:
  1) Ler planilha Excel "Portfolio_analitico.xlsx";
  2) Extrair e limpar as 'Author Keywords';
  3) Contar frequência e calcular porcentagem de ocorrência;
  4) Mostrar barra de progresso;
  5) Gerar gráfico Top-15 em PNG;
  6) Exportar relatório formatado em Excel (.xlsx);
  7) Registrar data/hora de início e término da execução.

USO NO CLUSTER UFSC (vLab JupyterLab ou SLURM):
  • Coloque este script e o arquivo "Portfolio_analitico.xlsx" no mesmo diretório.
  • No terminal do vLab (Launcher → Terminal), rode:
      module load python/3.10 pandas matplotlib xlsxwriter openpyxl tqdm
      python keyword_analitico.py
  • OU submeta um job SLURM sem GPU:
      #SBATCH --cpus-per-task=4
      python keyword_analitico.py

DEPENDÊNCIAS (serão instaladas automaticamente se ausentes):
  pandas, openpyxl, matplotlib, xlsxwriter, tqdm

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado  
Data: 01-mai-2025
"""

import sys
import subprocess
import re
from pathlib import Path
from collections import Counter
from datetime import datetime
from zoneinfo import ZoneInfo

# ─── 0) marca data/hora de início (fuso São Paulo) ──────────
SP_TZ = ZoneInfo("America/Sao_Paulo")
inicio = datetime.now(SP_TZ)
print(f"▶️ Início em: {inicio:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)")

# 1) Instalar dependências se faltarem
def instalar_dependencias():
    deps = ['pandas', 'openpyxl', 'matplotlib', 'xlsxwriter', 'tqdm']
    for pkg in deps:
        try:
            __import__(pkg)
        except ImportError:
            print(f"⚙️ Instalando {pkg}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
            import site; site.addsitedir(site.USER_SITE)
            __import__(pkg)

instalar_dependencias()

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# 2) Define diretório e arquivos
base_dir    = Path.cwd()
input_file  = base_dir / 'Portfolio_analitico.xlsx'
output_xlsx = base_dir / 'keyword_analitico.xlsx'
output_png  = base_dir / 'top15_keywords.png'

# 3) Verifica existência do arquivo de entrada
if not input_file.exists():
    sys.exit(f"⛔ Arquivo de entrada não encontrado: {input_file}")

# 4) Leitura e validação
df = pd.read_excel(input_file)
if 'Author Keywords' not in df.columns:
    sys.exit("⛔ Coluna 'Author Keywords' não encontrada.")
keywords_ser = df['Author Keywords'].dropna()

# 5) Função de simplificação
def simplify_keyword(kw: str) -> str:
    patterns = [r'aasb\s*\d*', r'aaoifi\s*', r'aaer\s*', r'aacsb\s*']
    kw = kw.strip()
    for pat in patterns:
        kw = re.sub(pat, '', kw, flags=re.IGNORECASE).strip()
    return kw.lower()

# 6) Extração e limpeza com barra de progresso
print("\n🔄 Processando palavras-chave…")
irrelevant = [r'^\d+$', r'^\d+[a-z]$', r'century', r'audit act', r'^[a-z]{1,2}$']
all_keywords = []
for entry in tqdm(keywords_ser, unit='linhas'):
    partes = re.split(r'[;,/|]+| - ', str(entry))
    for kw in partes:
        kw = re.sub(r'[^\w\s]', '', kw).strip().lower()
        if (not kw
            or any(re.search(p, kw) for p in irrelevant)
            or len(kw.split()) > 3):
            continue
        kw = simplify_keyword(kw)
        if not re.fullmatch(r'[a-z0-9 ]+', kw):
            continue
        all_keywords.append(kw)

# 7) Contagem e percentual
total = len(all_keywords)
if total == 0:
    sys.exit("⛔ Nenhuma palavra-chave válida após processamento.")
counts = Counter(all_keywords)

# 8) Monta DataFrame de saída
df_out = pd.DataFrame([
    {'Palavra-chave': k, 'Ocorrências': v, 'Porcentagem (%)': round(v/total*100, 2)}
    for k, v in counts.items()
])
df_out = df_out.sort_values('Ocorrências', ascending=False).reset_index(drop=True)

# 9) Saída no terminal (preview + total)
print("\n🎯 Primeiras 15 linhas do DataFrame:")
print(df_out.head(15).to_string(index=False))
print(f"\n🔢 Total de palavras-chave mapeadas: {total:,}".replace(',', '.'))

# 10) Gráfico Top-15
top15 = counts.most_common(15)
if top15:
    kws, vals = zip(*top15)
    plt.figure(figsize=(10, 6))
    plt.barh(kws, vals)
    plt.gca().invert_yaxis()
    plt.xlabel('Ocorrências')
    plt.title('Top 15 Palavras-Chave')
    plt.tight_layout()
    plt.savefig(output_png)
    plt.close()

# 11) Formatação e exportação Excel
df_out['Porcentagem (%)'] = df_out['Porcentagem (%)']\
    .map(lambda x: f"{x:.2f}".replace('.', ',') + '%')
with pd.ExcelWriter(output_xlsx, engine='xlsxwriter') as writer:
    df_out.to_excel(writer, index=False, sheet_name='Resumo')
    wb = writer.book
    ws = writer.sheets['Resumo']
    header_fmt = wb.add_format({'bold': True, 'align': 'center'})
    text_fmt   = wb.add_format({'align': 'left'})
    num_fmt    = wb.add_format({'num_format': '#,##0', 'align': 'right'})
    for col_idx, _ in enumerate(df_out.columns):
        ws.write(0, col_idx, df_out.columns[col_idx], header_fmt)
    ws.set_column('A:A', 40, text_fmt)
    ws.set_column('B:B', 15, num_fmt)
    ws.set_column('C:C', 15, text_fmt)

# ─── 12) Relatório final com data e duração ─────────────────
fim     = datetime.now(SP_TZ)
duracao = str(fim - inicio).split('.')[0]
print("\n📊 RELATÓRIO FINAL — FREQUÊNCIA DE PALAVRAS-CHAVE")
print("="*55)
print(f"→ Total mapeado          : {total}")
print(f"⏱️ Tempo de execução: {duracao}")
print(f"📅 Finalizado em: {fim:%d/%m/%Y %H:%M:%S} (Florianópolis | SC)")
print("="*55 + "\n")

▶️ Início em: 2025-05-03 09:06:59 (Florianópolis | SC)

🔄 Processando palavras-chave…


100%|██████████| 13614/13614 [00:00<00:00, 32530.73linhas/s]



🎯 Primeiras 15 linhas do DataFrame:
            Palavra-chave  Ocorrências  Porcentagem (%)
      earnings management         1029             1.50
    management accounting          791             1.16
               accounting          790             1.16
     corporate governance          571             0.84
                 business          568             0.83
management and accounting          534             0.78
                  finance          277             0.41
                economics          240             0.35
           sustainability          227             0.33
          risk management          212             0.31
               management          211             0.31
      financial reporting          189             0.28
                     ifrs          173             0.25
           accountability          161             0.24
              performance          158             0.23

🔢 Total de palavras-chave mapeadas: 68.375

📊 RELATÓRIO FINAL — FR

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Dicionario_contabilidade_gerencial.py

Script 3 para:
  1) Carregar planilha Excel "keyword_analitico.xlsx" com contagem de frequência;
  2) Classificar semanticamente cada palavra-chave em subáreas de Contabilidade Gerencial;
  3) Ajustar automaticamente descritores de cada categoria com base nas top keywords não classificadas;
  4) Gerar e formatar o dicionário temático em Excel "Dicionario_Contabilidade_Gerencial.xlsx" com duas colunas (Categoria e Palavras-chave);
  5) Salvar scores de similaridade em "debug_dicionario_scores.xlsx";
  6) Salvar sugestões de termos em "suggested_terms.xlsx".

USO NO CLUSTER UFSC (JupyterLab ou SLURM):
  • Instale dependências: pip install pandas openpyxl xlsxwriter torch transformers scikit-learn tqdm numpy nltk psutil
  • No cluster: module load python/3.10 pandas torch transformers scikit-learn tqdm openpyxl xlsxwriter nltk
  • Execute: python Dicionario_contabilidade_gerencial.py
  • Para SLURM, use o script fornecido no final.

IMPORTANTE:
  • A planilha "keyword_analitico.xlsx" deve ter colunas 'Palavra-chave' e uma coluna de frequência (nome com 'freq', 'count' ou 'ocorr').
  • Arquivo deve estar no mesmo diretório do script (/home/jovyan/Congresso UFSC2025/).
  • A saída do dicionário será ajustada para apenas "Categoria" e "Palavras-chave" com todas as palavras classificadas.

DEPENDÊNCIAS:
  pandas, openpyxl, xlsxwriter, torch, transformers, scikit-learn, tqdm, numpy, nltk, psutil

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado
Contribuidores: [Adicione seu nome, se aplicável]
Licença: MIT
Data: 02/05/2025
"""

import sys
import subprocess
import re
import os
import logging
import psutil
from pathlib import Path
from datetime import datetime
from zoneinfo import ZoneInfo
from collections import Counter, defaultdict

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Desativar TensorFlow CUDA e oneDNN
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # Desativa GPU
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Desativa oneDNN

# Instalar dependências
def instalar_dependencias():
    """Instala as dependências necessárias se não estiverem disponíveis."""
    deps = ['pandas', 'openpyxl', 'xlsxwriter', 'torch', 'transformers', 'scikit-learn', 'tqdm', 'numpy', 'nltk', 'psutil']
    for pkg in deps:
        mod = 'sklearn' if pkg == 'scikit-learn' else pkg
        try:
            __import__(mod)
            logger.info(f"{pkg} já instalado.")
        except ImportError:
            logger.info(f"Instalando {pkg}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
            import site
            site.addsitedir(site.USER_SITE)

try:
    instalar_dependencias()
except Exception as e:
    logger.error(f"Erro ao instalar dependências: {e}")
    sys.exit(1)

import numpy as np
import pandas as pd
import torch
try:
    from tqdm.notebook import tqdm  # Para JupyterLab
except ImportError:
    from tqdm import tqdm  # Fallback para terminal
import nltk
from nltk.stem import SnowballStemmer
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Inicializar stemmer
logger.info("Inicializando stemmer...")
stemmer = SnowballStemmer('english')

# Definir caminhos e timezone
base_dir = Path.cwd()
input_file = base_dir / 'keyword_analitico.xlsx'
output_file = base_dir / 'Dicionario_Contabilidade_Gerencial.xlsx'
debug_file = base_dir / 'debug_dicionario_scores.xlsx'
sugg_file = base_dir / 'suggested_terms.xlsx'

SC_TZ = ZoneInfo('America/Sao_Paulo')
inicio = datetime.now(SC_TZ)

# Exibir início
print(f"▶️ Início em: {inicio:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)")

# Perguntar sobre debug
debug_mode = input("Deseja ativar debug (s/n)? ").strip().lower() == 's'
if debug_mode:
    logger.setLevel(logging.DEBUG)
    logger.debug("Modo debug ativado.")

# Verificar arquivo de entrada
logger.info(f"Verificando arquivo: {input_file}")
if not input_file.exists():
    logger.error(f"Arquivo {input_file} não encontrado.")
    sys.exit(1)

# Carregar planilha
logger.info("Carregando planilha Excel...")
try:
    df = pd.read_excel(input_file)
    logger.info(f"Colunas encontradas: {list(df.columns)}")
except Exception as e:
    logger.error(f"Erro ao carregar planilha: {e}")
    sys.exit(1)

freq_cols = [c for c in df.columns if any(x in c.lower() for x in ['freq', 'count', 'ocorr'])]
if 'Palavra-chave' not in df.columns or not freq_cols:
    logger.error("A planilha deve ter colunas 'Palavra-chave' e frequência (com 'freq', 'count' ou 'ocorr').")
    sys.exit(1)
freq_col = freq_cols[0]
logger.info(f"Coluna de frequência: {freq_col}")

# Configurar dispositivo e modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"Usando device: {device}")
try:
    logger.info("Carregando tokenizer e modelo BERT...")
    tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    model = AutoModel.from_pretrained('bert-base-multilingual-cased').to(device)
    model.eval()
except Exception as e:
    logger.error(f"Erro ao carregar modelo BERT: {e}")
    sys.exit(1)

# Definir descritores das categorias (atualizado com as palavras sugeridas)
categorias = {
    'Avaliação de desempenho': (
        'performance management; performance evaluation; performance appraisal; performance measurement;'
        'kpi; roi; roa; bsc; balanced scorecard; benchmarking; efficiency analysis; value-based performance;'
        'economic value added; eva; employee performance; performance review; productivity metrics;'
        'performance feedback; goal setting; performance indicators; outcome measurement; performance tracking;'
        'performance standards; employee appraisal; performance assessment; organizational metrics;'
        'performance improvement'
    ),
    'Orçamento': (
        'budgeting; budget planning; budget forecast; budget participation; budgetary control; beyond budgeting;'
        'rolling forecast; zero-based budgeting; incremental budgeting; budget constraints; budget approval;'
        'budget cycle; budget preparation; budget allocation; budget review; budget oversight'
    ),
    'Custo': (
        'cost accounting; cost systems; abc costing; activity-based costing; absorption costing;'
        'variable costing; target costing; kaizen costing; standard costing; life cycle costing;'
        'cost control; cost analysis; cost estimate; cost drivers; cost behavior; '
        'direct costs; indirect costs; cost structure; unit cost'
    ),
    'Sistema de Controle Gerencial': (
        'management control systems; mcs; control systems; levers of control; belief systems;'
        'boundary systems; strategic control; operational control; internal control systems;'
        'accounting information systems; results control; action control; diagnostic control;'
        'interactive control; control mechanisms; control practices; control frameworks;'
        'management accounting systems; control policies; control feedback mechanisms; control processes;'
        'management oversight; control implementation'
    ),
    'Comportamento Organizacional': (
        'organizational behavior; employee engagement; motivation theories; cognitive bias;'
        'organizational culture; leadership styles; team dynamics; organizational commitment;'
        'psychological safety; work motivation; workplace behavior; job satisfaction;'
        'organizational justice; organizational psychology; affective commitment; job involvement;'
        'turnover intention; group behavior; interpersonal conflict; communication styles;'
        'power dynamics; organizational learning; diversity management; emotional intelligence;'
        'behavioral decision-making; team performance; workplace dynamics; employee motivation;'
        'organizational change; conflict resolution'
    ),
    'Educação Contábil': (
        'accounting education; curriculum development; teaching methods; learning strategies;'
        'pedagogy in higher education; professional development; continuing education; instructional design;'
        'accounting instructor; accounting teaching; educational innovation; didactic strategies;'
        'blended learning; teaching accounting; student academic performance; learning outcomes; capstone course;'
        'ethics education; teaching cases; IFRS education; audit education; tax education; critical thinking;'
        'student assessment; accounting skills; competency-based education; student engagement;'
        'accounting students; accounting pedagogy; accounting curriculum; student learning;'
        'accounting certification; accounting faculty; accounting courses; student accounting skills;'
        'accounting departments; academic accounting; accounting academics; accounting research education'
    ),
    'Outros Temas': (
        'general business trends; non-specific industry topics; miscellaneous organizational concepts;'
        'emerging technologies; social impact initiatives; global economic trends; cross-industry innovation;'
        'non-accounting academic research; societal trends; cultural studies; public administration;'
        'nonprofit management; social entrepreneurship; community development; human rights; gender equality;'
        'environmental policy; urban development; smart cities; systemic change'
    )
}

# Exibir categorias encontradas
print("📋 CATEGORIAS ENCONTRADAS NO DICIONÁRIO:")
print("----------------------------------------")
for i, cat in enumerate(sorted(categorias.keys()), 1):
    print(f"{i}. {cat.lower()}")
print("----------------------------------------")
logger.info("Categorias inicializadas.")

# Função de embedding
def embed_texts(texts, batch_size=16):
    """Gera embeddings para uma lista de textos usando BERT."""
    logger.info(f"Calculando embeddings para {len(texts)} textos...")
    mem = psutil.virtual_memory()
    logger.info(f"Memória disponível: {mem.available / (1024**3):.2f} GB")
    if mem.available < 1 * (1024**3):
        logger.warning("Memória baixa! Pode causar travamento.")
    embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Processando batches", unit="batch"):
        batch = texts[i:i + batch_size]
        try:
            enc = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            with torch.no_grad():
                out = model(**enc).last_hidden_state
            embs.append(out.mean(1).cpu().numpy())
            torch.cuda.empty_cache() if device.type == 'cuda' else None
        except Exception as e:
            logger.error(f"Erro no batch {i//batch_size + 1}: {e}")
            raise
    logger.info("Embeddings calculados com sucesso.")
    return np.vstack(embs)

# Processar palavras-chave
logger.info("Processando palavras-chave...")
unique_kws = sorted(df['Palavra-chave'].astype(str).unique())
logger.info(f"Encontradas {len(unique_kws)} palavras-chave únicas.")
stems = [' '.join(stemmer.stem(tok) for tok in re.findall(r"\w+", kw.lower())) for kw in unique_kws]
try:
    kw_emb = embed_texts(stems)
    kw_emb /= np.linalg.norm(kw_emb, axis=1, keepdims=True)
except Exception as e:
    logger.error(f"Erro ao calcular embeddings das palavras-chave: {e}")
    sys.exit(1)

# Embeddings das categorias
logger.info("Calculando embeddings das categorias...")
descs = list(categorias.values())
cat_keys = list(categorias.keys())
try:
    cat_emb = embed_texts(descs)
    cat_emb /= np.linalg.norm(cat_emb, axis=1, keepdims=True)
except Exception as e:
    logger.error(f"Erro ao calcular embeddings das categorias: {e}")
    sys.exit(1)

# Regras manuais de reclassificação baseadas na análise
reclass_rules = {
    '6g cellular communication': 'Outros Temas',
    '1c company': 'Sistema de Controle Gerencial',
    '401k investment': 'Orçamento',
    '2003 northeast blackout': 'Outros Temas',
    '10k report': 'Avaliação de desempenho',
    'a76 studies': 'Outros Temas',
    'team productivity': 'Comportamento Organizacional',
    'planning and budgeting': 'Orçamento',
    'audit committee': 'Outros Temas',
    'digital technologies': 'Outros Temas',
    'learning curve': 'Educação Contábil',
}

# Classificar palavras-chave
logger.info("Classificando palavras-chave...")
debug = []
cats_assigned = []
for idx, emb in enumerate(tqdm(kw_emb, desc='Classificando', unit='kw')):
    kw = unique_kws[idx]
    if kw in reclass_rules:
        cat = reclass_rules[kw]
    else:
        sims = cosine_similarity([emb], cat_emb)[0]
        imax = sims.argmax()
        cat = cat_keys[imax] if sims[imax] >= 0.3 else 'Não Classificado'  # Mantido limiar em 0.3
    cats_assigned.append(cat)
    debug.append({'keyword': kw, **{cat_keys[i]: float(sims[i]) for i in range(len(cat_keys))}, 'assigned': cat})

# Contar palavras por categoria
word_counts = Counter(cats_assigned)

# Salvar debug
logger.info("Salvando debug...")
try:
    debug_df = pd.DataFrame(debug)
    debug_df.to_excel(debug_file, index=False)
    logger.info(f"Debug salvo em: {debug_file}")
except Exception as e:
    logger.error(f"Erro ao salvar debug: {e}")
    sys.exit(1)

# Sugestões de termos
logger.info("Gerando sugestões...")
nao_cl = debug_df[debug_df.assigned == 'Não Classificado'][['keyword']].drop_duplicates()
freq_df = df[['Palavra-chave', freq_col]].rename(columns={'Palavra-chave': 'keyword', freq_col: 'frequency'})
sugg = nao_cl.merge(freq_df, on='keyword', how='left').sort_values('frequency', ascending=False).head(20)
try:
    sugg.to_excel(sugg_file, index=False)
    logger.info(f"Sugestões salvas em: {sugg_file}")
except Exception as e:
    logger.error(f"Erro ao salvar sugestões: {e}")
    sys.exit(1)

# Integrar sugestões
logger.info("Integrando sugestões...")
new_terms = defaultdict(list)
sugg_stems = [' '.join(stemmer.stem(tok) for tok in re.findall(r"\w+", kw.lower())) for kw in sugg.keyword]
try:
    sugg_emb = embed_texts(sugg_stems)
    sugg_emb /= np.linalg.norm(sugg_emb, axis=1, keepdims=True)
except Exception as e:
    logger.error(f"Erro ao calcular embeddings das sugestões: {e}")
    sys.exit(1)

for kw, emb in zip(sugg.keyword, sugg_emb):
    sims = cosine_similarity([emb], cat_emb)[0]
    imax = sims.argmax()
    cat = cat_keys[imax] if sims[imax] >= 0.3 else None  # Mantido limiar em 0.3
    if cat:
        new_terms[cat].append(kw)

# Atualizar descritores
logger.info("Atualizando descritores...")
for cat, terms in new_terms.items():
    if terms:
        categorias[cat] += '; ' + '; '.join(terms)

# Gerar dicionário com duas colunas
logger.info("Gerando dicionário final com duas colunas...")
try:
    # Criar dicionário com palavras classificadas
    cat_words = defaultdict(list)
    for kw, cat in zip(unique_kws, cats_assigned):
        cat_words[cat].append(kw)
    dict_df = pd.DataFrame([
        (cat, '; '.join(sorted(words))) for cat, words in cat_words.items()
    ], columns=['Categoria', 'Palavras-chave'])
    dict_df = dict_df.sort_values('Categoria')
    
    # Salvar no Excel
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        dict_df.to_excel(writer, index=False, sheet_name='Dicionário')
        wb = writer.book
        ws = writer.sheets['Dicionário']
        fmt = wb.add_format({'font_name': 'Times New Roman', 'font_size': 12, 'align': 'center', 'valign': 'vcenter'})
        ws.set_column('A:A', 25, fmt)
        ws.set_column('B:B', 80, fmt)
    logger.info(f"Dicionário salvo em: {output_file}")
except Exception as e:
    logger.error(f"Erro ao gerar dicionário: {e}")
    sys.exit(1)

# Relatório final
logger.info("Gerando relatório final...")
fim = datetime.now(SC_TZ)
dur = str(fim - inicio).split('.')[0]

print("📊 DICIONÁRIO FINAL ATUALIZADO")
for c, n in word_counts.items():
    print(f"- {c}: {n} keywords")
print(f"📅 Finalizado em: {fim:%d/%m/%Y %H:%M:%S} (Florianópolis | SC)")
print(f"⏱️ Tempo de execução: {dur}")
print(f"→ Dicionário salvo em: {output_file.name}")
logger.info("Script concluído com sucesso.")

2025-05-03 12:07:22,720 - INFO - pandas já instalado.
2025-05-03 12:07:22,721 - INFO - openpyxl já instalado.
2025-05-03 12:07:22,721 - INFO - xlsxwriter já instalado.
2025-05-03 12:07:23,813 - INFO - torch já instalado.
2025-05-03 12:07:23,998 - INFO - transformers já instalado.
2025-05-03 12:07:24,368 - INFO - scikit-learn já instalado.
2025-05-03 12:07:24,369 - INFO - tqdm já instalado.
2025-05-03 12:07:24,369 - INFO - numpy já instalado.
2025-05-03 12:07:24,601 - INFO - nltk já instalado.
2025-05-03 12:07:24,602 - INFO - psutil já instalado.
2025-05-03 12:07:24,857 - INFO - Inicializando stemmer...


▶️ Início em: 2025-05-03 09:07:24 (Florianópolis | SC)


Deseja ativar debug (s/n)?  s


2025-05-03 12:07:28,244 - DEBUG - Modo debug ativado.
2025-05-03 12:07:28,245 - INFO - Verificando arquivo: /home/jovyan/Congresso UFSC2025/keyword_analitico.xlsx
2025-05-03 12:07:28,245 - INFO - Carregando planilha Excel...
2025-05-03 12:07:28,759 - INFO - Colunas encontradas: ['Palavra-chave', 'Ocorrências', 'Porcentagem (%)']
2025-05-03 12:07:28,761 - INFO - Coluna de frequência: Ocorrências
2025-05-03 12:07:28,762 - INFO - Usando device: cpu
2025-05-03 12:07:28,762 - INFO - Carregando tokenizer e modelo BERT...
2025-05-03 12:07:30.318653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746274050.330247 2114694 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746274050.333490 2114694 cuda_blas.cc:1407] Unable to register cuBLAS factory: At

📋 CATEGORIAS ENCONTRADAS NO DICIONÁRIO:
----------------------------------------
1. avaliação de desempenho
2. comportamento organizacional
3. custo
4. educação contábil
5. orçamento
6. outros temas
7. sistema de controle gerencial
----------------------------------------


...

Classificando:   0%|          | 0/26125 [00:00<?, ?kw/s]

2025-05-03 12:08:24,434 - INFO - Salvando debug...
2025-05-03 12:08:26,384 - INFO - Debug salvo em: /home/jovyan/Congresso UFSC2025/debug_dicionario_scores.xlsx
2025-05-03 12:08:26,386 - INFO - Gerando sugestões...
2025-05-03 12:08:26,400 - INFO - Sugestões salvas em: /home/jovyan/Congresso UFSC2025/suggested_terms.xlsx
2025-05-03 12:08:26,400 - INFO - Integrando sugestões...
2025-05-03 12:08:26,401 - INFO - Calculando embeddings para 20 textos...
2025-05-03 12:08:26,401 - INFO - Memória disponível: 1751.91 GB


Processando batches:   0%|          | 0/2 [00:00<?, ?batch/s]

2025-05-03 12:08:26,551 - INFO - Embeddings calculados com sucesso.
2025-05-03 12:08:26,556 - INFO - Atualizando descritores...
2025-05-03 12:08:26,557 - INFO - Gerando dicionário final com duas colunas...
2025-05-03 12:08:26,577 - INFO - Dicionário salvo em: /home/jovyan/Congresso UFSC2025/Dicionario_Contabilidade_Gerencial.xlsx
2025-05-03 12:08:26,578 - INFO - Gerando relatório final...
2025-05-03 12:08:26,578 - INFO - Script concluído com sucesso.


📊 DICIONÁRIO FINAL ATUALIZADO
- Orçamento: 8008 keywords
- Sistema de Controle Gerencial: 2678 keywords
- Avaliação de desempenho: 2582 keywords
- Outros Temas: 2125 keywords
- Custo: 9310 keywords
- Educação Contábil: 865 keywords
- Não Classificado: 309 keywords
- Comportamento Organizacional: 248 keywords
📅 Finalizado em: 03/05/2025 09:08:26 (Florianópolis | SC)
⏱️ Tempo de execução: 0:01:01
→ Dicionário salvo em: Dicionario_Contabilidade_Gerencial.xlsx


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
classificar_artigos_tematica.py (Versão Ajustada Final)

Script 4 para:
  1) Normalização dos scores por número de descritores por categoria.
  2) Adição de n-gramas para capturar expressões compostas.
  3) Pré-processamento com remoção de stop words, pontuação e normalização.
  4) Otimização com índice invertido.
  5) Inclusão de debug para análise dos scores.

USO NO CLUSTER UFSC (JupyterLab ou SLURM):
  • Instale dependências: pip install pandas openpyxl nltk tqdm
  • No cluster: module load python/3.10 pandas openpyxl nltk
  • Execute: python classificar_artigos_tematica.py
  • Arquivos devem estar no mesmo diretório.

IMPORTANTE:
  • "Portfolio_analitico.xlsx" deve ter as colunas 'Year', 'Cited by', 'Title', 'Abstract', 'Affiliations', 'Source title', 'Author Keywords'.
  • "Dicionario_Contabilidade_Gerencial.xlsx" e "SJR 2023.xlsx" devem estar no diretório atual.
  • Saída será salva como "Portfolio_analitico_classificado.xlsx" com as abas 'Classificação' e 'Resumo_Categorias'.

DEPENDÊNCIAS:
  pandas, openpyxl, nltk, tqdm

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado
Revisões: Grok (xAI)
Data: 02/05/2025
Licença: MIT (Ciência Aberta - Open Science Framework)
"""

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
import re
from pathlib import Path
from datetime import datetime
from zoneinfo import ZoneInfo
from tqdm import tqdm

# Definir timezone e marcar início
SC_TZ = ZoneInfo('America/Sao_Paulo')
inicio = datetime.now(SC_TZ)

print(f"▶️ Início em: {inicio:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)")

# Inicializar NLTK e baixar os recursos necessários
print("Baixando recursos do NLTK...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("Recursos do NLTK baixados com sucesso.")

# Habilitar progresso com pandas
tqdm.pandas()

# Definir caminhos
base_dir = Path.cwd()
input_dict_file = base_dir / 'Dicionario_Contabilidade_Gerencial.xlsx'
input_sjr_file = base_dir / 'SJR 2023.xls'
input_artigos_file = base_dir / 'Portfolio_analitico.xlsx'
output_file = base_dir / 'Portfolio_analitico_classificado.xlsx'

# Carregar o dicionário temático
print(f"Carregando {input_dict_file}...")
try:
    dict_df = pd.read_excel(input_dict_file)
    categorias = dict_df.set_index('Categoria')['Palavras-chave'].to_dict()
    print(f"Dicionário carregado com {len(categorias)} categorias.")
except FileNotFoundError:
    print(f"Erro: Arquivo {input_dict_file} não encontrado.")
    exit(1)
except Exception as e:
    print(f"Erro ao carregar o dicionário: {e}")
    exit(1)

# Construir índice invertido para otimização
def build_inverted_index(categorias):
    index = {}
    palavras_por_categoria = {}
    for categoria, palavras in categorias.items():
        palavras_lista = re.split(r'[,;]\s*', palavras.lower())
        palavras_por_categoria[categoria] = len(palavras_lista)
        for palavra in palavras_lista:
            if palavra not in index:
                index[palavra] = []
            index[palavra].append(categoria)
    return index, palavras_por_categoria

inverted_index, palavras_por_categoria = build_inverted_index(categorias)

# Carregar o arquivo SJR 2023
print(f"Carregando {input_sjr_file}...")
try:
    sjr_df = pd.read_excel(input_sjr_file)
    if not all(col in sjr_df.columns for col in ['Title', 'Best Quartile']):
        print("Erro: A planilha 'SJR 2023.xls' deve ter as colunas 'Title' e 'Best Quartile'.")
        exit(1)
    sjr_dict = pd.Series(
        sjr_df['Best Quartile'].values,
        index=sjr_df['Title'].str.lower().str.strip()
    ).to_dict()
    print(f"SJR 2023 carregado com {len(sjr_dict)} entradas.")
except FileNotFoundError:
    print(f"Erro: Arquivo {input_sjr_file} não encontrado.")
    exit(1)
except Exception as e:
    print(f"Erro ao carregar o arquivo SJR: {e}")
    exit(1)

# Carregar a planilha de artigos
print(f"Carregando {input_artigos_file}...")
try:
    artigos_df = pd.read_excel(input_artigos_file)
    expected_cols = ['Year', 'Cited by', 'Title', 'Abstract', 'Affiliations', 'Source title', 'Author Keywords']
    if not all(col in artigos_df.columns for col in expected_cols):
        print(f"Erro: A planilha 'Portfolio_analitico.xlsx' deve ter as colunas {expected_cols}.")
        print(f"Colunas disponíveis: {artigos_df.columns.tolist()}")
        exit(1)
    print(f"Planilha de artigos carregada com {len(artigos_df)} linhas.")
except FileNotFoundError:
    print(f"Erro: Arquivo {input_artigos_file} não encontrado.")
    exit(1)
except Exception as e:
    print(f"Erro ao carregar os artigos: {e}")
    exit(1)

# Funções utilitárias
def extrair_pais(affiliations):
    if pd.isna(affiliations) or affiliations == '':
        return ''
    aff_text = str(affiliations).split(';')[0]
    if ',' in aff_text:
        pais = aff_text.split(',')[-1].strip()
        return pais if pais else 'Não Identificado'
    return 'Não Identificado'

# Pré-processamento de texto
stop_words = set(stopwords.words('english') + stopwords.words('portuguese'))
def preprocess_text(texto):
    texto = str(texto).lower()
    texto = re.sub(r'[^\w\s]', '', texto)  # Remover pontuação
    texto_tokens = word_tokenize(texto)
    texto_tokens = [t for t in texto_tokens if t not in stop_words]
    # Adicionar bigramas
    bigrams = [' '.join(ng) for ng in ngrams(texto_tokens, 2)]
    texto_tokens.extend(bigrams)
    return texto_tokens

# Função de classificação com debug
def classificar_artigo(texto):
    texto_tokens = preprocess_text(texto)
    categoria_counts = {}
    for token in texto_tokens:
        if token in inverted_index:
            for categoria in inverted_index[token]:
                categoria_counts[categoria] = categoria_counts.get(categoria, 0) + 1
    scores = {}
    melhor_categoria = None
    max_taxa = 0
    min_correspondencias = 2  # Limiar mínimo
    for categoria, count in categoria_counts.items():
        if count >= min_correspondencias:
            taxa = count / palavras_por_categoria[categoria]
            scores[categoria] = taxa
            if taxa > max_taxa:
                max_taxa = taxa
                melhor_categoria = categoria
        else:
            scores[categoria] = 0
    return melhor_categoria if melhor_categoria else 'Não Classificado', scores

# Processamento das colunas
print("Extraindo países da coluna 'Affiliations'...")
artigos_df['País'] = artigos_df['Affiliations'].progress_apply(extrair_pais)

print("Associando 'Best Quartile' com base no SJR 2023...")
artigos_df['Best Quartile'] = (
    artigos_df['Source title']
    .str.lower().str.strip()
    .map(sjr_dict)
    .fillna('Não Encontrado')
)

print("Classificando artigos por temática...")
resultados = artigos_df.progress_apply(
    lambda row: classificar_artigo(
        ' '.join([str(row['Title']), str(row['Abstract']), str(row['Author Keywords'])])
    ), axis=1
)
artigos_df['Grupo Temático'] = [res[0] for res in resultados]
artigos_df['Debug_Scores'] = [str(res[1]) for res in resultados]

# Reorganizar colunas
final_columns = ['Year', 'Cited by', 'Title', 'Abstract', 'Affiliations', 'País',
                 'Source title', 'Best Quartile', 'Author Keywords', 'Grupo Temático', 'Debug_Scores']
artigos_df = artigos_df[final_columns]

# Salvar resultados
print(f"Salvando resultado em {output_file}...")
try:
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        artigos_df.to_excel(writer, index=False, sheet_name='Classificação')
        resumo_df = (
            artigos_df['Grupo Temático']
            .value_counts()
            .rename_axis('Grupo Temático')
            .reset_index(name='Contagem')
        )
        resumo_df.to_excel(writer, index=False, sheet_name='Resumo_Categorias')
    print("\nResumo de artigos por categoria:")
    print(resumo_df.to_string(index=False))

    fim = datetime.now(SC_TZ)
    dur = fim - inicio
    horas, resto = divmod(dur.seconds, 3600)
    minutos, segundos = divmod(resto, 60)
    print(f"\n📅 Finalizado em: {fim:%d/%m/%Y %H:%M:%S} (Florianópolis | SC)")
    print(f"⏱️ Tempo de execução: {horas}h {minutos}m {segundos}s")
    print(f"→ Classificação salva em: {output_file.name}")
except Exception as e:
    print(f"Erro ao salvar a classificação: {e}")
    exit(1)

▶️ Início em: 2025-05-03 09:10:07 (Florianópolis | SC)
Baixando recursos do NLTK...
Recursos do NLTK baixados com sucesso.
Carregando /home/jovyan/Congresso UFSC2025/Dicionario_Contabilidade_Gerencial.xlsx...
Dicionário carregado com 8 categorias.
Carregando /home/jovyan/Congresso UFSC2025/SJR 2023.xls...
SJR 2023 carregado com 1576 entradas.
Carregando /home/jovyan/Congresso UFSC2025/Portfolio_analitico.xlsx...
Planilha de artigos carregada com 13614 linhas.
Extraindo países da coluna 'Affiliations'...


100%|██████████| 13614/13614 [00:00<00:00, 794573.84it/s]


Associando 'Best Quartile' com base no SJR 2023...
Classificando artigos por temática...


100%|██████████| 13614/13614 [00:06<00:00, 2029.88it/s]


Salvando resultado em /home/jovyan/Congresso UFSC2025/Portfolio_analitico_classificado.xlsx...

Resumo de artigos por categoria:
               Grupo Temático  Contagem
                    Orçamento      3485
             Não Classificado      2691
                        Custo      2025
 Comportamento Organizacional      1364
                 Outros Temas      1268
Sistema de Controle Gerencial      1258
      Avaliação de desempenho       933
            Educação Contábil       590

📅 Finalizado em: 03/05/2025 09:10:18 (Florianópolis | SC)
⏱️ Tempo de execução: 0h 0m 10s
→ Classificação salva em: Portfolio_analitico_classificado.xlsx


In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
coerencia_comunicacional.py

Script 5 para:
  1) Ler Portfolio_analitico_classificado.xlsx e preservar dados & formatação da coluna J;
  2) Calcular score de coerência título vs. resumo com SBERT em batch;
  3) Gerar histogramas e boxplot;
  4) Destacar 1% de piores coerências nas colunas C e D;
  5) Restaurar coluna J, formatar planilha e salvar em Portfolio_analitico_coerencia.xlsx;
  6) Exibir relatório final com contagens, tempo e data de execução.

USO NO CLUSTER UFSC (JupyterLab ou SLURM):
  • Carregue GPU e Python 3.10:
      module load cuda/11.7 python/3.10
  • Execute:
      python coerencia_comunicacional.py

DEPENDÊNCIAS (serão instaladas automaticamente):
  sentence-transformers, pandas, openpyxl, tqdm, matplotlib

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado
Data: 02/05/2025
"""

import sys, subprocess
from pathlib import Path
from datetime import datetime
from zoneinfo import ZoneInfo
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
import matplotlib.pyplot as plt

# 0) Instala deps se necessário
def instalar(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
for pkg in ("sentence-transformers","pandas","openpyxl","tqdm","matplotlib"):
    try:
        __import__(pkg.replace('-', '_'))
    except ImportError:
        print(f"⚙️ Instalando {pkg}...")
        instalar(pkg)

# --- INÍCIO (fuso São Paulo) -------------------------------------
SP_TZ       = ZoneInfo("America/Sao_Paulo")
data_inicio = datetime.now(SP_TZ)
print(f"▶️ Início em: {data_inicio:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)\n")

# 1) Caminhos de entrada e saída
def get_paths():
    try:
        base = Path(__file__).resolve().parent
    except NameError:
        base = Path.cwd()
    inp = base / "Portfolio_analitico_classificado.xlsx"
    out = base / "Portfolio_analitico_coerencia.xlsx"
    if not inp.exists():
        sys.exit(f"⛔ Arquivo não encontrado: {inp}")
    return inp, out
EXCEL_IN, EXCEL_OUT = get_paths()

# 2) Carrega DataFrame e preserva coluna J
def carregar_planilha(path):
    df = pd.read_excel(path)
    wb = load_workbook(path)
    ws = wb.active
    backup_j = {}
    for row in range(2, ws.max_row+1):
        cell = ws[f"J{row}"]
        backup_j[row] = {'value': cell.value, 'fill': cell.fill.copy() if cell.fill and cell.fill.fill_type else None}
    wb.close()
    return df, backup_j

# 3) Calcular coerência corretamente usando título vs. abstract
def calcular_coerencia(df):
    device = 'cuda' if util.torch.cuda.is_available() else 'cpu'
    print(f"▶️ Usando device: {device}\n")
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device=device)
    titles    = df['Title'].fillna('').astype(str).tolist()
    abstracts = df['Abstract'].fillna('').astype(str).tolist()
    print("⏳ Gerando embeddings dos títulos...")
    emb_titles = model.encode(titles, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
    print("⏳ Gerando embeddings dos resumos...")
    emb_abstracts = model.encode(abstracts, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
    print("⚡ Calculando similaridades título vs. resumo...")
    cosines = util.cos_sim(emb_titles, emb_abstracts).diag().cpu().numpy()
    faixas = []
    for s in cosines:
        if s >= 0.80:   faixas.append('Alta')
        elif s >= 0.60: faixas.append('Média')
        else:           faixas.append('Baixa')
    df['Coerência (Score)']          = cosines
    df['Classificação de Coerência'] = faixas
    return df, cosines, faixas

# 4) Gera e salva plots
def plot_distribuicoes(scores):
    valid = [s for s in scores if not np.isnan(s)]
    plt.figure(figsize=(8,5))
    plt.hist(valid, bins=30, edgecolor='black')
    plt.title('Distribuição dos Scores de Coerência')
    plt.xlabel('Score de Similaridade')
    plt.ylabel('Frequência')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('Histograma_Coerencia.png', dpi=300, bbox_inches='tight')
    plt.close()
    plt.figure(figsize=(6,5))
    plt.boxplot(valid, vert=False)
    plt.title('Boxplot dos Scores de Coerência')
    plt.xlabel('Score de Similaridade')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('Boxplot_Coerencia.png', dpi=300, bbox_inches='tight')
    plt.close()

# 5) Salva novo Excel com backup coluna J
def salvar_com_backup(df, backup):
    df.to_excel(EXCEL_OUT, index=False)
    wb = load_workbook(EXCEL_OUT)
    ws = wb.active
    for row, info in backup.items():
        cell = ws[f"J{row}"]
        cell.value = info['value']
        if info['fill']:
            cell.fill = info['fill']
    return wb, ws

# 6) Destacar 1% piores coerências
def destacar_menores(df, ws):
    valid = df[df['Coerência (Score)'].notnull()]
    n = max(1, int(0.01 * len(valid)))
    worst = valid.nsmallest(n, 'Coerência (Score)').index.tolist()
    fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
    for idx in worst:
        row = idx + 2
        ws[f"C{row}"].fill = fill
        ws[f"D{row}"].fill = fill

# 7) Formatação geral
def formatar(ws):
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row,
                             min_col=1, max_col=ws.max_column):
        for c in row:
            c.font = Font(name='Calibri', size=11)
            c.alignment = Alignment(horizontal='center', vertical='center')
    for c in ws[1]: c.font = Font(bold=True)

# 8) Relatório final
def relatorio_final(df, faixas, start):
    total = len(df)
    cnt = {k: faixas.count(k) for k in ['Alta','Média','Baixa']}
    none = total - sum(cnt.values())
    now  = datetime.now(SP_TZ)
    dur  = str(now - start).split('.')[0]
    fmt = lambda x: f"{x:,d}".replace(',', '.')
    print("\n===== RELATÓRIO FINAL =====")
    print(f"Total analisados       : {fmt(total)}")
    print(f"Alta Coerência         : {fmt(cnt['Alta'])} ({cnt['Alta']/total*100:.1f}%)")
    print(f"Média Coerência        : {fmt(cnt['Média'])} ({cnt['Média']/total*100:.1f}%)")
    print(f"Baixa Coerência        : {fmt(cnt['Baixa'])} ({cnt['Baixa']/total*100:.1f}%)")
    print(f"Sem dados/Erro         : {fmt(none)} ({none/total*100:.1f}%)")
    print(f"Tempo de execução: {dur}")
    print(f"Finalizado em: {now:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)")

# === MAIN ===
if __name__ == '__main__':
    df, backup = carregar_planilha(EXCEL_IN)
    df, scores, faixas = calcular_coerencia(df)
    plot_distribuicoes(scores)
    wb, ws = salvar_com_backup(df, backup)
    destacar_menores(df, ws)
    formatar(ws)
    wb.save(EXCEL_OUT)
    wb.close()
    relatorio_final(df, faixas, data_inicio)

▶️ Início em: 2025-05-03 09:12:56 (Florianópolis | SC)



2025-05-03 12:12:59,748 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


▶️ Usando device: cpu

⏳ Gerando embeddings dos títulos...


Batches:   0%|          | 0/213 [00:00<?, ?it/s]

⏳ Gerando embeddings dos resumos...


Batches:   0%|          | 0/213 [00:00<?, ?it/s]

⚡ Calculando similaridades título vs. resumo...

===== RELATÓRIO FINAL =====
Total analisados       : 13.614
Alta Coerência         : 1.179 (8.7%)
Média Coerência        : 8.573 (63.0%)
Baixa Coerência        : 3.862 (28.4%)
Sem dados/Erro         : 0 (0.0%)
Tempo de execução: 0:01:37
Finalizado em: 2025-05-03 09:14:33 (Florianópolis | SC)


In [21]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
h1ab_regressao_OLS_orcamento.py

Script 6 para:
  - Executar análise OLS das hipóteses H1a/H1b com controle por país, ano e quartil.
  - Exporta:
  - Tabela de regressão (limpa) com significância no estilo R (*, **, ***).
  - Heatmap dos Top 20 países (Coerência por País e Ano).
  - Estatísticas descritivas por país.

Inclui print técnico do modelo sem coeficientes de país e com nomes legíveis.

USO NO CLUSTER UFSC (JupyterLab ou SLURM):
  • Instale dependências: pip install pandas statsmodels openpyxl seaborn matplotlib
  • No cluster: module load python/3.10 pandas statsmodels openpyxl seaborn matplotlib
  • Execute: python h1ab_regressao_OLS_orcamento.py
  • Arquivo "Portfolio_analitico_coerencia.xlsx" deve estar no diretório atual.

DEPENDÊNCIAS:
  pandas, statsmodels, openpyxl, seaborn, matplotlib

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado
Contribuidores: [Adicione seu nome, se aplicável]
Licença: MIT (Ciência Aberta - Open Science Framework)
Data: 02/05/2025
"""

import sys
import subprocess
from pathlib import Path
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import re

# Função para instalar pacotes, se necessário
def instalar(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("pandas", "statsmodels", "openpyxl", "seaborn", "matplotlib"):
    try:
        __import__(pkg.replace('-', '_'))
    except ImportError:
        print(f"⚙️ Instalando {pkg}...")
        instalar(pkg)

# Função para marcar significância no estilo R
def marcar_sig(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return ''

# Função para limpar nomes das variáveis
def limpar_nome_variavel(v):
    if 'Grupo Temático' in v:
        if 'T.' in v:
            return v.split('T.')[1].strip(']')
        else:
            return 'Orçamento'  # ALTERAÇÃO: Mudei de 'Avaliação de Desempenho' pra 'Orçamento'
    elif v == 'Intercept':
        return 'Intercepto'
    elif v == 'Q("Year")':
        return 'Ano'
    elif v == 'Q("Best Quartile")':
        return 'Quartil'
    return v

# Início
SC_TZ = ZoneInfo("America/Sao_Paulo")
inicio = datetime.now(SC_TZ)
print(f"▶️ Início em: {inicio:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)\n")

# Leitura do arquivo
arquivo = Path("Portfolio_analitico_coerencia.xlsx")
if not arquivo.exists():
    sys.exit(f"⛔ Arquivo não encontrado: {arquivo}")
df = pd.read_excel(arquivo)

# Preparação dos dados
df = df[['Coerência (Score)', 'Grupo Temático', 'Year', 'País', 'Best Quartile']].dropna()
df = df[df['Best Quartile'].str.startswith('Q')]
df['Best Quartile'] = df['Best Quartile'].str.replace('Q', '').astype(int)
df = df[df['País'].str.isalpha() & ~df['País'].isin(['Não Identificado'])]

# Regressão OLS
print("📊 Rodando regressão OLS (H1a/b)...")
# ALTERAÇÃO: Adicionei Treatment("Orçamento") pra definir Orçamento como referência
formula = 'Q("Coerência (Score)") ~ C(Q("Grupo Temático"), Treatment("Orçamento")) + Q("Year") + C(Q("País")) + Q("Best Quartile")'
modelo = smf.ols(formula=formula, data=df).fit()

# Tabela de saída formatada no estilo R
tabela = modelo.summary2().tables[1].reset_index()
# Removemos o filtro para incluir todas as variáveis (não apenas Grupo Temático)
tabela = tabela[~tabela['index'].str.contains('País')].copy()  # Exclui apenas os coeficientes de País
tabela.columns = ['Variável', 'Coeficiente', 'Erro Padrão', 't', 'p-valor', 'IC 95% inferior', 'IC 95% superior']
tabela['Variável'] = tabela['Variável'].apply(limpar_nome_variavel)
tabela['Sig'] = tabela['p-valor'].apply(marcar_sig)
tabela = tabela[['Variável', 'Coeficiente', 'Erro Padrão', 't', 'p-valor', 'Sig', 'IC 95% inferior', 'IC 95% superior']]
tabela = tabela.round(4)
tabela.to_excel("H1ab_Regressao_OLS_Resultados.xlsx", index=False)

# Imprimir a tabela no console
print("\n📋 Tabela de Regressão OLS (H1a/b):")
print(tabela.to_string(index=False))

# Resumo técnico limpo (sem coeficientes de país e com nomes legíveis)
print("\n📊 Resumo técnico do modelo OLS (sem coef. de país):\n")
resumo_str = modelo.summary().as_text()
resumo_limpo = re.sub(
    r'C\(Q\("País"\)\)\[T\..+?\]\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\n',
    '', resumo_str)

# Substituir nomes técnicos por nomes legíveis
substituir_variaveis = {
    'C\\(Q\\("Grupo Temático"\\)\\)\\[T.Comportamento Organizacional\\]': "Comportamento Organizacional",
    'C\\(Q\\("Grupo Temático"\\)\\)\\[T.Custo\\]': "Custo",
    'C\\(Q\\("Grupo Temático"\\)\\)\\[T.Educação Contábil\\]': "Educação Contábil",
    'C\\(Q\\("Grupo Temático"\\)\\)\\[T.Não Classificado\\]': "Não Classificado",
    'C\\(Q\\("Grupo Temático"\\)\\)\\[T.Orçamento\\]': "Orçamento",
    'C\\(Q\\("Grupo Temático"\\)\\)\\[T.Outros Temas\\]': "Outros Temas",
    'C\\(Q\\("Grupo Temático"\\)\\)\\[T.Sistema de Controle Gerencial\\]': "Sistema de Controle Gerencial",
    'Q\\("Year"\\)': "Ano",
    'Q\\("Best Quartile"\\)': "Quartil",
    'Intercept': 'Intercepto'
}
for padrao, nome_limpo in substituir_variaveis.items():
    resumo_limpo = re.sub(padrao, nome_limpo, resumo_limpo)

# Adicionar nota sobre significância
resumo_limpo += "\nSignificance levels: * p<0.05, ** p<0.01, *** p<0.001"
print(resumo_limpo)

# Heatmap dos Top 20 países
print("\n📈 Gerando heatmap com Top 20 países...")
df_heat = df[['País', 'Year', 'Coerência (Score)']].dropna()
df_heat = df_heat[df_heat['Year'].between(1995, 2025)]
top20 = df_heat['País'].value_counts().nlargest(20).index.tolist()
df_heat = df_heat[df_heat['País'].isin(top20)]
pivot = df_heat.groupby(['País', 'Year'])['Coerência (Score)'].mean().reset_index().pivot(
    index='País', columns='Year', values='Coerência (Score)'
)
pivot = pivot.reindex(columns=range(1995, 2026))
plt.figure(figsize=(18, 10))
sns.heatmap(pivot, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=0.5, linecolor='gray')
plt.title("Média do Score de Coerência por País × Ano (Top 20 Países)")
plt.xlabel("Ano")
plt.ylabel("País")
plt.tight_layout()
plt.savefig("H1ab_Heatmap_Top20_Paises.png", dpi=300)
plt.close()

# Estatísticas descritivas por país
print("📊 Gerando estatísticas por país...")
agg = df.groupby('País')['Coerência (Score)'].agg(['count', 'mean', 'std', 'median', 'min', 'max']).round(3)
agg.columns = ['N Artigos', 'Média', 'Desvio Padrão', 'Mediana', 'Mínimo', 'Máximo']
agg = agg.sort_values('Média', ascending=False)
agg.to_excel("H1ab_Resumo_Por_Pais.xlsx")

# Relatório final
fim = datetime.now(SC_TZ)
dur = str(fim - inicio).split('.')[0]
print("\n📁 Arquivos gerados:")
print("  • H1ab_Regressao_OLS_Resultados.xlsx")
print("  • H1ab_Heatmap_Top20_Paises.png")
print("  • H1ab_Resumo_Por_Pais.xlsx")
print("\n===== RELATÓRIO FINAL =====")
print(f"Tempo de execução: {dur}")
print(f"Finalizado em: {fim:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)")

▶️ Início em: 2025-05-04 10:13:47 (Florianópolis | SC)

📊 Rodando regressão OLS (H1a/b)...

📋 Tabela de Regressão OLS (H1a/b):
                     Variável  Coeficiente  Erro Padrão       t  p-valor Sig  IC 95% inferior  IC 95% superior
                   Intercepto      -1.3833       0.5001 -2.7659   0.0057  **          -2.3637          -0.4029
      Avaliação de desempenho      -0.0035       0.0061 -0.5686   0.5697              -0.0154           0.0085
 Comportamento Organizacional      -0.0218       0.0051 -4.2418   0.0000 ***          -0.0318          -0.0117
                        Custo      -0.0233       0.0047 -4.9910   0.0000 ***          -0.0324          -0.0141
            Educação Contábil      -0.0021       0.0074 -0.2762   0.7824              -0.0166           0.0125
             Não Classificado      -0.0095       0.0043 -2.2137   0.0269   *          -0.0179          -0.0011
                 Outros Temas       0.0027       0.0054  0.4996   0.6174              -0.0079   

In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
h2ab_regressao_OLS_analise_completa.py

Script 7 para:
  - Executar análise OLS das hipóteses H2a/H2b, investigando a relação entre o impacto
    dos periódicos ((Q1-Q2) vs. (Q3-Q4)) e a coerência comunicacional, controlando por
    país e ano. Exporta:
  - Tabela de regressão com significância no estilo R (*, **, ***).
  - Heatmap dos Top 20 países (Coerência por Ano).
  - Estatísticas descritivas por país.

USO NO CLUSTER UFSC (JupyterLab ou SLURM):
  • Instale dependências: pip install pandas statsmodels openpyxl seaborn matplotlib
  • No cluster: module load python/3.10 pandas statsmodels openpyxl seaborn matplotlib
  • Execute: python h2ab_regressao_OLS_analise_completa.py
  • Arquivo "Portfolio_analitico_coerencia.xlsx" deve estar no diretório atual.

DEPENDÊNCIAS:
  pandas, statsmodels, openpyxl, seaborn, matplotlib

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado
Licença: MIT (Ciência Aberta - Open Science Framework)
Data: 02/05/2025
"""

import sys
import subprocess
from pathlib import Path
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import re

# Função para instalar pacotes, se necessário
def instalar(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("pandas", "statsmodels", "openpyxl", "seaborn", "matplotlib"):
    try:
        __import__(pkg.replace('-', '_'))
    except ImportError:
        print(f"⚙️ Instalando {pkg}...")
        instalar(pkg)

# Função para marcar significância no estilo R
def marcar_sig(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return ''

# Função para limpar nomes das variáveis
def limpar_nome_variavel(v):
    if v == 'Intercept':
        return 'Intercepto'
    elif v == 'Q("Year")':
        return 'Ano'
    elif v == 'C(Q("Quartil_Alto"))[T.1]':
        return '(Q1-Q2)'
    return v

# Função para extrair o número do quartil para ordenação
def extrair_quartil(v):
    if v == 'Intercepto':
        return -2  # Intercepto primeiro
    elif v == 'Ano':
        return -1  # Ano depois do Intercepto
    elif v == '(Q1-Q2)':
        return 1  # (Q1-Q2) primeiro
    elif v == '(Q3-Q4)':
        return 2  # (Q3-Q4) depois
    return 0  # Outros casos (não deve ocorrer)

# Início
SC_TZ = ZoneInfo("America/Sao_Paulo")
inicio = datetime.now(SC_TZ)
print(f"▶️ Início em: {inicio:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)\n")

# Leitura do arquivo
arquivo = Path("Portfolio_analitico_coerencia.xlsx")
if not arquivo.exists():
    sys.exit(f"⛔ Arquivo não encontrado: {arquivo}")
df = pd.read_excel(arquivo)

# Preparação dos dados
df = df[['Coerência (Score)', 'Best Quartile', 'Year', 'País']].dropna()
df = df[df['Best Quartile'].str.startswith('Q')]
df['Best Quartile'] = df['Best Quartile'].str.replace('Q', '').astype(int)
df = df[df['País'].str.isalpha() & ~df['País'].isin(['Não Identificado'])]

# Criar dummy para Quartil Alto (Q1-Q2 = 1, Q3-Q4 = 0)
df['Quartil_Alto'] = (df['Best Quartile'] <= 2).astype(int)

# Regressão OLS
print("📊 Rodando regressão OLS (H2a/b)...")
formula = 'Q("Coerência (Score)") ~ C(Q("Quartil_Alto")) + Q("Year") + C(Q("País"))'
modelo = smf.ols(formula=formula, data=df).fit()

# Tabela de saída formatada no estilo R
tabela = modelo.summary2().tables[1].reset_index()
tabela = tabela[~tabela['index'].str.contains('País')].copy()  # Exclui apenas os coeficientes de País
tabela.columns = ['Variável', 'Coeficiente', 'Erro Padrão', 't', 'p-valor', 'IC 95% inferior', 'IC 95% superior']
tabela['Variável'] = tabela['Variável'].apply(limpar_nome_variavel)
print("🔍 Variáveis antes da ordenação:", tabela['Variável'].tolist())  # Log para depuração
tabela['Sig'] = tabela['p-valor'].apply(marcar_sig)
# Adicionar linha de referência (Q3-Q4) manualmente
ref_row = pd.DataFrame({
    'Variável': ['(Q3-Q4)'],
    'Coeficiente': [0.0000],
    'Erro Padrão': [0.0000],
    't': [0.0000],
    'p-valor': [1.0000],
    'IC 95% inferior': [0.0000],
    'IC 95% superior': [0.0000],
    'Sig': ['']
})
tabela = pd.concat([tabela, ref_row], ignore_index=True)
# Reordenar para listar Intercepto, Ano, (Q1-Q2), (Q3-Q4)
tabela['Ordem'] = tabela['Variável'].apply(extrair_quartil)
tabela = tabela.sort_values('Ordem')
tabela = tabela[['Variável', 'Coeficiente', 'Erro Padrão', 't', 'p-valor', 'Sig', 'IC 95% inferior', 'IC 95% superior']]
tabela = tabela.round(4)
tabela.to_excel("H2ab_Regressao_OLS_Resultados.xlsx", index=False)

# Imprimir a tabela no console
print("\n📋 Tabela de Regressão OLS (H2a/b):")
print("Dependente: Coerência (Score)")
print("Nota: (Q3-Q4) é a referência (coeficiente 0).")
print(tabela.to_string(index=False))

# Resumo técnico limpo (sem coeficientes de país e com nomes legíveis)
print("\n📊 Resumo técnico do modelo OLS (sem coef. de país):\n")
resumo_str = modelo.summary().as_text()
resumo_limpo = re.sub(
    r'C\(Q\("País"\)\)\[T\..+?\]\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\s+[-\d\.e\+]+\n',
    '', resumo_str)
substituir_variaveis = {
    r'C\(Q\("Quartil_Alto"\)\)\[T\.1\]': "(Q1-Q2)",
    r'Q\("Year"\)': "Ano",
    r'Intercept': 'Intercepto'
}
for padrao, nome_limpo in substituir_variaveis.items():
    resumo_limpo = re.sub(padrao, nome_limpo, resumo_limpo)
resumo_limpo = re.sub(r'Dep\. Variable:.*\n', 'Dep. Variable: Coerência (Score)\n', resumo_limpo)
resumo_limpo += "\nSignificance levels: * p<0.05, ** p<0.01, *** p<0.001"
print(resumo_limpo)

# Heatmap dos Top 20 países
print("\n📈 Gerando heatmap com Top 20 países...")
df_heat = df[['País', 'Year', 'Coerência (Score)']].dropna()
df_heat = df_heat[df_heat['Year'].between(1995, 2025)]
top20 = df_heat['País'].value_counts().nlargest(20).index.tolist()
df_heat = df_heat[df_heat['País'].isin(top20)]
pivot = df_heat.groupby(['País', 'Year'])['Coerência (Score)'].mean().reset_index().pivot(
    index='País', columns='Year', values='Coerência (Score)'
)
pivot = pivot.reindex(columns=range(1995, 2026))
plt.figure(figsize=(18, 10))
sns.heatmap(pivot, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=0.5, linecolor='gray')
plt.title("Média do Score de Coerência por País × Ano (Top 20 Países)")
plt.xlabel("Ano")
plt.ylabel("País")
plt.tight_layout()
plt.savefig("H2ab_Heatmap_Top20_Paises.png", dpi=300)
plt.close()

# Estatísticas descritivas por país
print("📊 Gerando estatísticas por país...")
agg = df.groupby('País')['Coerência (Score)'].agg(['count', 'mean', 'std', 'median', 'min', 'max']).round(3)
agg.columns = ['N Artigos', 'Média', 'Desvio Padrão', 'Mediana', 'Mínimo', 'Máximo']
agg = agg.sort_values('Média', ascending=False)
agg.to_excel("H2ab_Resumo_Por_Pais.xlsx")

# Relatório final
fim = datetime.now(SC_TZ)
dur = str(fim - inicio).split('.')[0]
print("\n📁 Arquivos gerados:")
print("  • H2ab_Regressao_OLS_Resultados.xlsx")
print("  • H2ab_Heatmap_Top20_Paises.png")
print("  • H2ab_Resumo_Por_Pais.xlsx")
print("\n===== RELATÓRIO FINAL =====")
print(f"Tempo de execução: {dur}")
print(f"Finalizado em: {fim:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)")

▶️ Início em: 2025-05-04 14:06:06 (Florianópolis | SC)

📊 Rodando regressão OLS (H2a/b)...
🔍 Variáveis antes da ordenação: ['Intercepto', '(Q1-Q2)', 'Ano']

📋 Tabela de Regressão OLS (H2a/b):
Dependente: Coerência (Score)
Nota: (Q3-Q4) é a referência (coeficiente 0).
  Variável  Coeficiente  Erro Padrão       t  p-valor Sig  IC 95% inferior  IC 95% superior
Intercepto      -1.2666       0.4988 -2.5394   0.0111   *          -2.2444          -0.2889
       Ano       0.0009       0.0002  3.7948   0.0001 ***           0.0004           0.0014
   (Q1-Q2)       0.0207       0.0035  5.9960   0.0000 ***           0.0140           0.0275
   (Q3-Q4)       0.0000       0.0000  0.0000   1.0000               0.0000           0.0000

📊 Resumo técnico do modelo OLS (sem coef. de país):

                              OLS Regression Results                              
Dep. Variable: Coerência (Score)
Model:                                OLS   Adj. R-squared:                  0.047
Method:            

In [27]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
h3ab_regressao_OLS_analise_completa_corrigido.py

Script 8 para:
  - Executar análise de Regressão OLS para as hipóteses H3a/H3b, investigando a relação
    entre a coerência comunicacional (Classificação de Coerência: Alta vs. Média/Baixa)
    e o número de citações (log(1 + Cited by)), controlando por Best Quartile e Ano.
    Exporta:
  - Tabela de regressão OLS com significância no estilo R (*, **, ***), com variáveis de controle separadas e descrições detalhadas.
  - Estatísticas descritivas por Classificação de Coerência.
  - Resumo técnico do modelo.

USO NO CLUSTER UFSC (JupyterLab ou SLURM):
  • Instale dependências: pip install pandas statsmodels openpyxl numpy scipy
  • No cluster: module load python/3.10 pandas statsmodels openpyxl numpy scipy
  • Execute: python h3ab_regressao_OLS_analise_completa_corrigido.py
  • Arquivo "Portfolio_analitico_coerencia.xlsx" deve estar no diretório atual.

DEPENDÊNCIAS:
  pandas, statsmodels, openpyxl, numpy, scipy

Autor: G.O., Renato | NEIMAC | PPGC | UFSC | Mestrado
Licença: MIT (Ciência Aberta - Open Science Framework)
Data: 03/05/2025
"""

import sys
import subprocess
from pathlib import Path
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy.stats as stats
import re

# Função para instalar pacotes, se necessário
def instalar(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("pandas", "statsmodels", "openpyxl", "numpy", "scipy"):
    try:
        __import__(pkg.replace('-', '_'))
    except ImportError:
        print(f"⚙️ Instalando {pkg}...")
        instalar(pkg)

# Função para marcar significância no estilo R
def marcar_sig(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return ''

# Função para limpar nomes das variáveis e adicionar descrições
def limpar_nome_variavel_com_descricao(v):
    if v == 'Intercept':
        return 'Intercepto'
    elif v == 'Q("Year")':
        return 'Ano'
    elif v.startswith('C(Q("Classificação de Coerência"))[T.'):
        categoria = v.split('T.')[1].split(']')[0]
        if categoria == 'Alta':
            return 'Alta (coerência ≥ 0,80)'
        elif categoria == 'Baixa':
            return 'Baixa (coerência < 0,60)'
        elif categoria == 'Média':
            return 'Média (coerência entre 0,60 e < 0,80)'
    elif v.startswith('C(Q("Best Quartile"))[T.'):
        return f"Quartil {v.split('T.')[1].split(']')[0]}"
    return v

# Função para extrair ordem das variáveis e identificar se é de controle
def extrair_ordem_variavel(v):
    if v == 'Intercepto':
        return -2, True
    elif v == 'Ano':
        return -1, True
    elif v.startswith('Alta'):
        return 1, False
    elif v.startswith('Baixa'):
        return 2, False
    elif v.startswith('Média'):
        return 3, False
    elif v.startswith('Quartil'):
        if v == 'Quartil NE':
            return 4, True
        try:
            return 4 + int(v.split(' ')[1][1:]), True  # Extrai número de "Q1", "Q2", etc.
        except (ValueError, IndexError):
            return 4, True  # Trata valores não numéricos como NE
    return 100, False

# Início
SC_TZ = ZoneInfo("America/Sao_Paulo")
inicio = datetime.now(SC_TZ)
print(f"▶️ Início em: {inicio:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)\n")

# Leitura do arquivo
arquivo = Path("Portfolio_analitico_coerencia.xlsx")
if not arquivo.exists():
    sys.exit(f"⛔ Arquivo não encontrado: {arquivo}")
df = pd.read_excel(arquivo)

# Preparação dos dados
print("📊 Preparando dados...")
df = df[['Cited by', 'Classificação de Coerência', 'Best Quartile', 'Year']]
df['Best Quartile'] = df['Best Quartile'].fillna('NE').replace(['-', 'Não Encontrado'], 'NE')
df['Classificação de Coerência'] = df['Classificação de Coerência'].fillna('Média')
df['Year'] = df['Year'].fillna(df['Year'].median())
df['Cited by'] = df['Cited by'].fillna(0).astype(int)

# Tratamento de valores extremos em Cited by (winsorização no top 5%)
print("🔍 Tratando valores extremos em Cited by...")
cited_by_winsorized = stats.mstats.winsorize(df['Cited by'], limits=[0, 0.05])
df['Cited by'] = cited_by_winsorized
df['Log_Cited_by'] = np.log1p(df['Cited by'])

# Verificação de dados
print("🔍 Verificando dados...")
print(f"  • Observações: {len(df)}")
print(f"  • Zeros em Cited by: {100 * (df['Cited by'] == 0).mean():.1f}%")
print(f"  • Categorias em Classificação de Coerência: {df['Classificação de Coerência'].value_counts().to_dict()}")
print(f"  • Categorias em Best Quartile: {df['Best Quartile'].value_counts().to_dict()}")
for col in ['Best Quartile', 'Classificação de Coerência']:
    rare_cats = df[col].value_counts()[df[col].value_counts() < 100].index
    if len(rare_cats) > 0:
        print(f"  • Aviso: Categorias raras em {col} (<100 observações): {list(rare_cats)}")

# Regressão OLS (modelo principal)
print("📊 Rodando regressão OLS (H3a/b)...")
formula_ols = 'Q("Log_Cited_by") ~ C(Q("Classificação de Coerência")) + C(Q("Best Quartile")) + Q("Year")'
modelo_ols = smf.ols(formula=formula_ols, data=df).fit(cov_type='HC3')

# Tabela de resultados (apenas OLS)
print("\n📋 Tabela de regressão OLS (H3a/b):")
tabela_ols = modelo_ols.summary2().tables[1].reset_index()
tabela_ols['Variável'] = tabela_ols['index'].apply(limpar_nome_variavel_com_descricao)
print("🔍 Colunas em tabela_ols:", tabela_ols.columns.tolist())
p_value_col_ols = 'P>|t|' if 'P>|t|' in tabela_ols.columns else 'P>|z|' if 'P>|z|' in tabela_ols.columns else 'pvalue'
stat_col_ols = 't' if 't' in tabela_ols.columns else 'z'
tabela_ols['Sig'] = tabela_ols[p_value_col_ols].apply(marcar_sig)

# Adicionar a categoria de referência (Alta)
alta_row = pd.DataFrame({
    'index': ['C(Q("Classificação de Coerência"))[T.Alta]'],
    'Coef.': [0.0],
    'Std.Err.': [0.0],
    stat_col_ols: [0.0],
    p_value_col_ols: [1.0],
    'Sig': [''],
    '[0.025': [0.0],
    '0.975]': [0.0],
    'Variável': ['Alta (coerência ≥ 0,80)']
})
tabela_ols = pd.concat([alta_row, tabela_ols], ignore_index=True)

# Identificar se é variável de controle
tabela_ols[['Ordem', 'É Controle']] = tabela_ols['Variável'].apply(lambda x: pd.Series(extrair_ordem_variavel(x)))

# Separar variáveis independentes e de controle
tabela_independentes = tabela_ols[tabela_ols['É Controle'] == False].sort_values('Ordem')
tabela_controle = tabela_ols[tabela_ols['É Controle'] == True].sort_values('Ordem')

# Concatenar as partes sem a linha divisória com NaN
tabela = pd.concat([tabela_independentes, tabela_controle], ignore_index=True)

# Finalizar a tabela
tabela = tabela[['Variável', 'Coef.', 'Std.Err.', stat_col_ols, p_value_col_ols, 'Sig', '[0.025', '0.975]']]
tabela.columns = ['Variável', 'Coeficiente', 'Erro Padrão', 'z/t', 'p-valor', 'Sig', 'IC 95% inferior', 'IC 95% superior']
tabela = tabela.round(4)
tabela.to_excel("H3ab_Regressao_Comparativa_Resultados.xlsx", index=False)

# Imprimir a tabela no console
print("Dependente: log(1 + Cited by) (OLS)")
print(tabela.to_string(index=False))

# Resumo técnico limpo
print("\n📊 Resumo técnico do modelo:\n")
resumo_ols = modelo_ols.summary().as_text()
substituir_variaveis = {
    r'C\(Q\("Classificação de Coerência"\)\)\[T\.([^\]]+)\]': r'\1',
    r'Q\("Year"\)': "Ano",
    r'Intercept': 'Intercepto',
    r'C\(Q\("Best Quartile"\)\)\[T\.([^\]]+)\]': r'Quartil \1'
}
resumo_ols_limpo = resumo_ols
for padrao, nome_limpo in substituir_variaveis.items():
    resumo_ols_limpo = re.sub(padrao, nome_limpo, resumo_ols_limpo)
resumo_ols_limpo = re.sub(r'Dep\. Variable:.*\n', 'Dep. Variable: log(1 + Cited by)\n', resumo_ols_limpo)
resumo_combined = f"Modelo OLS:\n{resumo_ols_limpo}\nSignificance levels: * p<0.05, ** p<0.01, *** p<0.001"
print(resumo_combined)

# Estatísticas descritivas por Classificação de Coerência
print("\n📊 Gerando estatísticas por Classificação de Coerência...")
agg = df.groupby('Classificação de Coerência')['Cited by'].agg(['count', 'mean', 'std', 'median', 'min', 'max']).round(3)
agg.columns = ['N Artigos', 'Média', 'Desvio Padrão', 'Mediana', 'Mínimo', 'Máximo']
agg = agg.sort_values('Média', ascending=False)
agg.to_excel("H3ab_Resumo_Por_Coerencia.xlsx")

# Relatório final
fim = datetime.now(SC_TZ)
dur = str(fim - inicio).split('.')[0]
print("\n📁 Arquivos gerados:")
print("  • H3ab_Regressao_Comparativa_Resultados.xlsx")
print("  • H3ab_Resumo_Por_Coerencia.xlsx")
print("\n===== RELATÓRIO FINAL =====")
print(f"Tempo de execução: {dur}")
print(f"Finalizado em: {fim:%Y-%m-%d %H:%M:%S} (Florianópolis | SC)")

▶️ Início em: 2025-05-04 15:55:02 (Florianópolis | SC)

📊 Preparando dados...
🔍 Tratando valores extremos em Cited by...
🔍 Verificando dados...
  • Observações: 13614
  • Zeros em Cited by: 15.5%
  • Categorias em Classificação de Coerência: {'Média': 8573, 'Baixa': 3862, 'Alta': 1179}
  • Categorias em Best Quartile: {'Q1': 4721, 'NE': 3101, 'Q2': 3090, 'Q3': 1621, 'Q4': 1081}
📊 Rodando regressão OLS (H3a/b)...

📋 Tabela de regressão OLS (H3a/b):
🔍 Colunas em tabela_ols: ['index', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]', 'Variável']
Dependente: log(1 + Cited by) (OLS)
                             Variável  Coeficiente  Erro Padrão      z/t  p-valor Sig  IC 95% inferior  IC 95% superior
              Alta (coerência ≥ 0,80)       0.0000       0.0000   0.0000    1.000               0.0000           0.0000
             Baixa (coerência < 0,60)      -0.2019       0.0382  -5.2847    0.000 ***          -0.2768          -0.1270
Média (coerência entre 0,60 e < 0,80)      -0.0599